In [ ]:
path = "c:/Users/Nathalie/Nextcloud/LADi/Orthografie Trainer/Code/04_bias_mitigation/"

import sys

sys.path.insert(1, path + "classes")

from DecisionTree_insession import in_session_decision_tree
from KNN_insession import in_session_KNN
from MLP_insession import in_session_MLP
from MetricsClass import Evaluation

import pandas as pd
import seaborn as sns
import pickle

Historical Bias Mitigation: Drop Gender feature from training data

In [ ]:
# define features cols - no gender feature
feature_cols = [
        "Erstloesung",
        "Schussel",
        "Erfolg",
        "Schwierigkeit",
        "ist_Schulzeit",
        "MehrfachFalsch",
        "vorher_abgebrochen",
        "Fehler",
        "Klassenstufe",
        "Jahredabei",
        "AnzahlAufgaben",
        "Testposition__pruefung",
        "Testposition__training",
        "Testposition__version",
        "Art__GK",
        "Art__GR",
        "Art__GZ",
        "Art__K",
        "Art__LB",
        "UserAttribut",
        "OrderNumber",
        "steps",
    ]

Decision Tree Classifier

In [ ]:
DTE = in_session_decision_tree()
DTE.set_range(2, 4)  # 61

# set sex as column names 
DTE.set_minority_group("Sex__w")
DTE.set_majority_group("Sex__m")
DTE.set_demographic_category("gender")

# parameter False, as not balanced out, as representational bias is not attempted to be mitigated yet
DTE.load_matrices(
    "02_historical_bias_mitigation/gender_historical/matrix", ".pkl", False
)

DTE.set_feature_cols(
    feature_cols
)
DTE.set_minority_group("girls")
DTE.set_majority_group("boys")
metrics_DTE = DTE.loop_matrices("gender_historical/matrix",5,1,2)

In [ ]:
metrics_DTE

KNN

In [ ]:
KNN = in_session_KNN()
KNN.set_range(2, 3)  # 61

# set sex as column names for minority / majority group
KNN.set_minority_group("Sex__w")
KNN.set_majority_group("Sex__m")
KNN.set_demographic_category("gender")

# not balanced out, as representational bias is not attempted to be mitigated yet
KNN.load_matrices(
    "02_historical_bias_mitigation/gender_historical/matrix", ".pkl", False
)
KNN.set_feature_cols(
    feature_cols
)
KNN.set_minority_group("girls")
KNN.set_majority_group("boys")
metrics_KNN = KNN.loop_matrices("gender_historical/matrix",2,'uniform')

In [ ]:
metrics_KNN

MLP

In [ ]:
MLP = in_session_MLP()
MLP.set_range(2, 4)  # 61

# set sex as column names 
MLP.set_minority_group("Sex__w")
MLP.set_majority_group("Sex__m")
MLP.set_demographic_category("gender")

# parameter False, as not balanced out, as representational bias is not attempted to be mitigated yet
MLP.load_matrices(
    "02_historical_bias_mitigation/gender_historical/matrix", ".pkl", False
)
MLP.set_feature_cols(
    feature_cols
)
MLP.set_minority_group("girls")
MLP.set_majority_group("boys")
metrics_MLP = MLP.loop_matrices("gender_historical/matrix",'Adam','binary_crossentropy','accuracy',22,22)

In [ ]:
metrics_MLP

In [ ]:
## construct dfs for all groups from metric df
metrics = pd.concat([metrics_DTE, metrics_KNN, metrics_MLP])
grouped = metrics.groupby(metrics.group)
df_all = grouped.get_group("all")

In [ ]:
## plot accuracy by n and model
ax = sns.lineplot(data=df_all, x="Sentence", y="Accuracy", hue="model")

In [ ]:
## plot count of sentences by n and model
ax = sns.lineplot(data=df_all, x="Sentence", y="Length", hue="model")

Evaluate

In [ ]:
# metrics.to_pickle('test.pkl')
# infile = open('test.pkl', "rb")
# metrics = pickle.load(infile)
# infile.close()

In [ ]:
# infile = open('../../02_dropout_prediction/01_keep_it_up/all_metrics.pkl', "rb")
# metrics = pickle.load(infile)
# infile.close()
# metrics = metrics.reset_index()

In [ ]:
# evaluate metrics for fairness
evaluation_obj = Evaluation(metrics)
evaluation_obj.set_demographic_category("gender")
evaluation_obj.set_minority_group("girls")
evaluation_obj.set_majority_group("boys")
df_gender = evaluation_obj.evaluate()

In [ ]:
# show table
table = evaluation_obj.showTable(df_gender)
table

In [ ]:
## save results
writer = pd.ExcelWriter("df_gender_historical.xlsx", engine="xlsxwriter")
df_gender.to_excel(writer, sheet_name="Gender")
writer.save()